In [1]:
import pandas as pd
import gzip
import json
import seaborn as sns
import numpy as np
import time
import html
import re
import os
from typing import *
import urllib.request
from PIL import Image
import cv2 as cv
from matplotlib import pyplot as plt
from tqdm import tqdm, trange
tqdm.pandas()

In [ ]:
# 

In [2]:
%%time
metadata_test = pd.read_csv('metadata_parsed_with_images.csv')

Wall time: 8.47 s


In [3]:
metadata_test = metadata_test.iloc[0:100000]

In [4]:
metadata_test.shape

(100000, 12)

In [5]:
metadata_test = metadata_test.fillna('')

In [6]:
import ast

In [7]:
%%time
metadata_test['images'] = metadata_test['images'].apply(lambda x: ast.literal_eval(x))

Wall time: 780 ms


In [8]:
%%time
metadata_test['imagesHighRes'] = metadata_test['imagesHighRes'].apply(lambda x: ast.literal_eval(x))

Wall time: 704 ms


In [45]:
part = metadata_test.copy()

In [46]:
import traceback
from functools import partial

def get_image(urls, path='images_0/'):
    import traceback
    import urllib.request
    try:
        if len(urls) == 0:
            return None
        url = urls[0]
        urllib.request.urlretrieve(url, path + url[49:])
    except Exception:
        print(f"Exception: {traceback.format_exc()}")
    return None

In [47]:
import psutil, humanize

def mem_report():
    print("CPU RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ))

mem_report()

CPU RAM Free: 4.4 GB


In [48]:
def load_image(urls, path='images_0/'):
    import cv2 as cv
    try:
        if len(urls) == 0:
            return None
        url = urls[0]
        img = cv.imread(path + url[49:])
        return img
    except Exception:
        print(f"Exception: {traceback.format_exc()}")
        return False

In [49]:
import mapply

In [50]:
part["loaded_images"] = part.images.apply(partial(load_image, path='images_0/'))

In [51]:
part.shape

(100000, 13)

In [52]:
part = part[['rank_scaled', 'category', 'category_id', 'images', 'brand', 'price']]

In [53]:
import traceback

def image_url(urls, path='images_0/'):
    try:
        if len(urls) == 0:
            return ''
        url = urls[0]
        return path + url[49:]
    except Exception:
        print(f'{traceback.format_exc()}')
        return ''

part['image'] = part.images.apply(partial(image_url, path='images_0/'))

In [54]:
part = part.drop('images', axis=1)

In [55]:
part = part[part['image'] != '']

In [56]:
part.shape

(41278, 6)

In [57]:
from sklearn.model_selection import GroupShuffleSplit

gss = GroupShuffleSplit(test_size=.33, n_splits=1, random_state = 7).split(part, groups=part['category_id'])

X_train_inds, X_test_inds = next(gss)

train_data = part.iloc[X_train_inds]
test_data = part.iloc[X_test_inds]

In [58]:
print(train_data.shape)
print(test_data.shape)

(31057, 6)
(10221, 6)


In [2]:
import tensorflow as tf
import tensorflow_ranking as tfr

In [3]:
batch_size = 32
img_height = 180
img_width = 180
AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
tf.image.resize

In [91]:
train_data.head()

,rank_scaled,category,category_id,brand,price,image
0,1.953892,"Arts, Crafts & Sewing > Craft Supplies > Soap ...",1,,35.00,images_0/41pw8f%2BTF1L._SS40_.jpg
1,2.589388,"Arts, Crafts & Sewing > Craft Supplies > Soap ...",1,,35.00,images_0/41kr9Cq4RTL._SS40_.jpg
6,1.979594,"Arts, Crafts & Sewing > Craft Supplies > Soap ...",1,pinkie,8.99,images_0/41r8YYY0KrL._SS40_.jpg
10,2.630039,"Arts, Crafts & Sewing > Craft Supplies > Soap ...",1,pinkie,9.99,images_0/41l3EbmhxvL._SS40_.jpg
13,0.574717,Home & Kitchen (See Top 100 in Home & Kitchen),0,Roxy Display,33.95,images_0/41852QXCmyL._SS40_.jpg


In [92]:
metadata_tf_train = tf.data.Dataset.from_tensor_slices(train_data.to_dict(orient="list"))
metadata_tf_valid = tf.data.Dataset.from_tensor_slices(test_data.to_dict(orient="list"))

In [93]:
metadata_tf = metadata_tf_train

In [4]:
img_height = 180
img_width = 180

normalization_layer = tf.keras.layers.Rescaling(1./255)

img = tf.io.read_file('images_0/41pw8f%2BTF1L._SS40_.jpg')
img = tf.io.decode_jpeg(img, channels=3)
print(img.shape)
# normalization_layer(img)

(40, 40, 3)


In [5]:
img = tf.image.resize(img, [img_height, img_width])

In [7]:
img.shape

TensorShape([180, 180, 3])

In [8]:
np.array(img).astype(np.uint8).reshape(180 * 180 * 3)

array([218, 216, 217, ..., 196, 196, 196], dtype=uint8)

In [ ]:
ima

In [ ]:
item_price = example_features["item_price"] 
            image_count = example_features["image_count"]
            image_shape = example_features["image_shape"]
            image = example_features["image"]

In [7]:
tf.io.encode_jpeg(np.array(img).astype(np.uint8)).numpy()

b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x01,\x01,\x00\x00\xff\xdb\x00C\x00\x02\x01\x01\x01\x01\x01\x02\x01\x01\x01\x02\x02\x02\x02\x02\x04\x03\x02\x02\x02\x02\x05\x04\x04\x03\x04\x06\x05\x06\x06\x06\x05\x06\x06\x06\x07\t\x08\x06\x07\t\x07\x06\x06\x08\x0b\x08\t\n\n\n\n\n\x06\x08\x0b\x0c\x0b\n\x0c\t\n\n\n\xff\xdb\x00C\x01\x02\x02\x02\x02\x02\x02\x05\x03\x03\x05\n\x07\x06\x07\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\xff\xc0\x00\x11\x08\x00\xb4\x00\xb4\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98\x9

In [5]:
tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[tf.io.encode_jpeg(img).numpy()])
)

NameError: name 'value' is not defined

In [13]:
tf.train.BytesList(value=[tf.io.encode_jpeg(np.zeros([180, 180, 3])).numpy()])

value: "\377\330\377\340\000\020JFIF\000\001\001\001\001,\001,\000\000\377\333\000C\000\002\001\001\001\001\001\002\001\001\001\002\002\002\002\002\004\003\002\002\002\002\005\004\004\003\004\006\005\006\006\006\005\006\006\006\007\t\010\006\007\t\007\006\006\010\013\010\t\n\n\n\n\n\006\010\013\014\013\n\014\t\n\n\n\377\333\000C\001\002\002\002\002\002\002\005\003\003\005\n\007\006\007\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\377\300\000\021\010\000\264\000\264\003\001\"\000\002\021\001\003\021\001\377\304\000\037\000\000\001\005\001\001\001\001\001\001\000\000\000\000\000\000\000\000\001\002\003\004\005\006\007\010\t\n\013\377\304\000\265\020\000\002\001\003\003\002\004\003\005\005\004\004\000\000\001}\001\002\003\000\004\021\005\022!1A\006\023Qa\007\"q\0242\201\221\241\010#B\261\301\025R\321\360$3br\202\t\n\026\027\030\031\032%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\203\204\205\206\207\210\211\212\222\223\224\225\226\22

In [95]:
# normalization_layer = tf.keras.layers.Rescaling(1./255)

# def process_path(file_path):
#     img = tf.io.read_file(file_path)
#     img = tf.io.decode_jpeg(img, channels=3)
#     img = tf.image.resize(img, [img_height, img_width])
#     return normalization_layer(img)

In [96]:
# data = metadata_tf.map(lambda x: {
#     "category_id": x["category_id"],
#     "category": x["category"],
#     "image": process_path(x["image"]),
#     "brand": x["brand"],
#     "price": x["price"],
#     "rank": x["rank_scaled"]
# }, num_parallel_calls=AUTOTUNE)

In [105]:
# for x in data.take(1):
#     for key, value in x.items():
#         print(f"Example value of {key}: {value}")
#         print()

In [106]:
# categories = metadata_tf.map(lambda x: x["category"])

In [107]:
# categories_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
# categories_vocabulary.adapt(categories.batch(100))

In [112]:
# создаем списки из элементов с одинаковой категорией
# list_size = 40

# key_func = lambda x: categories_vocabulary(x["category"])
# reduce_func = lambda key, dataset: dataset.batch(list_size)

# ds = data.group_by_window(key_func=key_func, reduce_func=reduce_func, window_size=list_size)

# for x in ds.take(1):
#     for key, value in x.items():
#         print(f"Shape of {key}: {value.shape}")
#         print(f"Example value of {key}: {value[:3].numpy()}")
#         print()

In [113]:
# def _features_and_labels(x: Dict[str, tf.Tensor]) -> Tuple[Dict[str, tf.Tensor], tf.Tensor]:
#     labels = x.pop("rank")
#     return x, labels

# ds = ds.map(_features_and_labels)

# ds = ds.apply(tf.data.experimental.dense_to_ragged_batch(batch_size=32))

# посмотрим на размеры одного батча
# в нём 32 примера, каждый пример - отранжированный список в рамках одной категории с фичами
# for x, label in ds.take(1):
#     for key, value in x.items():
#         print(f"Shape of {key}: {value.shape}")
#         print(f"Example values of {key}: {value[:3, :3].numpy()}")
#         print()
#     print(f"Shape of label: {label.shape}")
#     print(f"Example values of label: {label[:3, :3].numpy()}")

In [117]:
# посмотрим на размеры одного батча
# в нём 32 примера, каждый пример - отранжированный список в рамках одной категории с фичами
for x, label in ds.take(1):
    for key, value in x.items():
        print(f"Shape of {key}: {value.shape}")
        print(f"Example values of {key}: {value[:3, :3].numpy()}")
        print()
    print(f"Shape of label: {label.shape}")
    print(f"Example values of label: {label[:3, :3].numpy()}")

Shape of category_id: (32, None)
Example values of category_id: [[8 8 8]
 [4 4 4]
 [4 4 4]]

Shape of category: (32, None)
Example values of category: [[b'Office Products (See top 100)' b'Office Products (See top 100)'
  b'Office Products (See top 100)']
 [b'Arts, Crafts & Sewing (See Top 100 in Arts, Crafts & Sewing)'
  b'Arts, Crafts & Sewing (See Top 100 in Arts, Crafts & Sewing)'
  b'Arts, Crafts & Sewing (See Top 100 in Arts, Crafts & Sewing)']
 [b'Arts, Crafts & Sewing (See Top 100 in Arts, Crafts & Sewing)'
  b'Arts, Crafts & Sewing (See Top 100 in Arts, Crafts & Sewing)'
  b'Arts, Crafts & Sewing (See Top 100 in Arts, Crafts & Sewing)']]

Shape of image: (32, None, 180, 180, 3)
Example values of image: [[[[[1.         1.         1.        ]
    [1.         1.         1.        ]
    [1.         1.         1.        ]
    ...
    [1.         1.         1.        ]
    [1.         1.         1.        ]
    [1.         1.         1.        ]]

   [[1.         1.         1.       

In [211]:
def prepare_data(metadata_tf):
    normalization_layer = tf.keras.layers.Rescaling(1./255)

    def process_path(file_path):
        img = tf.io.read_file(file_path)
        img = tf.io.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [img_height, img_width])
        return normalization_layer(img)

    data = metadata_tf.map(lambda x: {
        "category_id": x["category_id"],
        "category": x["category"],
        "image": process_path(x["image"]),
        "brand": x["brand"],
        "price": x["price"],
        "rank": x["rank_scaled"]
    }, num_parallel_calls=AUTOTUNE)

    categories = metadata_tf.map(lambda x: x["category"])

    categories_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
    categories_vocabulary.adapt(categories.batch(100))

    # создаем списки из элементов с одинаковой категорией
    list_size = 40

    key_func = lambda x: categories_vocabulary(x["category"])
    reduce_func = lambda key, dataset: dataset.batch(list_size)

    ds = data.group_by_window(key_func=key_func, reduce_func=reduce_func, window_size=list_size)
    
    

    def _features_and_labels(x: Dict[str, tf.Tensor]) -> Tuple[Dict[str, tf.Tensor], tf.Tensor]:
        labels = x.pop("rank")
        return x, labels

    ds = ds.map(_features_and_labels)

    ds = ds.apply(tf.data.experimental.dense_to_ragged_batch(batch_size=32))
#     ds = ds.apply(
#     lambda x: 
#         {
#         "price": x["price"].to_tensor(default_value=0, shape=[32, 40]), 
#         "image": x["image"].to_tensor(default_value=0, shape=[32, 40, 180, 180, 3]), 
#         "category_id": x["category_id"].to_tensor(default_value=0, shape=[32, 40]),
#         "label": x["label"].to_tensor(default_value=0, shape=[32, 40]),
#         }
#     )
    return ds

In [212]:
ds_train = prepare_data(metadata_tf_train)
ds_valid = prepare_data(metadata_tf_valid)

TypeError: 'BatchDataset' object is not subscriptable

In [208]:
class ImageEmbeddings(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(32, 3, activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(32, 3, activation='relu')
        self.conv3 = tf.keras.layers.Conv2D(32, 3, activation='relu')
        self.maxpool = tf.keras.layers.MaxPooling2D()
        self.flatten = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(128, activation='relu')
        self.price_dense = tf.keras.layers.Dense(128, activation='relu')
        self.category_dense = tf.keras.layers.Dense(128, activation='relu')
        self.d = tf.keras.layers.Dense(1, activation='relu')
        
    def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:        
        # На вход features - Dict
        
        # "category_id": (batch, list size) // first dim - batch size, second dim - ordered list size
        # "category": (batch, list size)
        # "image": (batch, list size, 180, 180, 3)
        # "brand": (batch, list size)
        # "price": (batch, list size)
        
        category_id = features["category_id"]
        image = features["image"]
        price = features["price"]
        
        category_id = category_id.to_tensor(default_value=0, shape=[32, 40])
        category_id = tf.expand_dims(category_id, axis=-1)
        
        price = price.to_tensor(default_value=0, shape=[32, 40])
        price = tf.expand_dims(price, axis=-1)
        
        image = image.to_tensor(default_value=0, shape=[32, 40, 180, 180, 3])
        shape = image.shape
        
        image = tf.reshape(image, [shape[0] * shape[1], shape[2], shape[3], shape[4]])
        print(image.shape)
        img = self.conv1(image)
        print(img.shape)
        img = self.maxpool(img)
        print(img.shape)
        img = self.conv2(img)
        print(img.shape)
        img = self.maxpool(img)
        print(img.shape)
        img = self.conv3(img)
        img = self.maxpool(img)
        img = self.flatten(img)
        
        # img: (batch * list size, 128)
        img = self.dense(img)
        print(img.shape)
        
        category_id = self.category_dense(category_id)
        price = self.price_dense(price)
        
        print(f"Category id shape: {category_id.shape}")
        print(f"Price shape: {price.shape}")
        
        img = tf.reshape(img, [32, 40, 128])
        print(f"Image shape: {img.shape}")
        
        cat = img * category_id
        price = img * price
        print("cat")
        print(cat.shape)
        print("price")
        print(price.shape)
        
         # concat: (None, None, 2)
        concat = tf.concat([cat, price], axis=-1)
        print(concat.shape)
    
        # output: (None, None, 1)
        output = self.d(concat)
        print(output.shape)
        output = tf.squeeze(output, axis=2)
        print(output.shape)
        return output

In [209]:
def train_and_eval_model(Model, ds_train, ds_valid, epochs=10, loss=tfr.keras.losses.ApproxNDCGLoss(), metrics=[tfr.keras.metrics.NDCGMetric()]):
    model = Model()
    optimizer = tf.keras.optimizers.Adam()
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    model.fit(ds_train, epochs=epochs)
    model.evaluate(ds_valid)
    return model

In [210]:
model = train_and_eval_model(ImageEmbeddings, ds_train, ds_valid)

Epoch 1/10
(1280, 180, 180, 3)
(1280, 178, 178, 32)
(1280, 89, 89, 32)
(1280, 87, 87, 32)
(1280, 43, 43, 32)
(1280, 128)
Category id shape: (32, 40, 128)
Price shape: (32, 40, 128)
Image shape: (32, 40, 128)
cat
(32, 40, 128)
price
(32, 40, 128)
(32, 40, 256)
(32, 40, 1)
(32, 40)


TypeError: in user code:

    File "C:\Users\artyo\anaconda3\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\artyo\anaconda3\lib\site-packages\tensorflow_ranking\python\keras\losses.py", line 196, in call  *
        losses, weights = self._loss.compute_unreduced_loss(
    File "C:\Users\artyo\anaconda3\lib\site-packages\tensorflow_ranking\python\losses_impl.py", line 617, in compute_unreduced_loss  *
        labels, logits, _, mask = self._prepare_and_validate_params(
    File "C:\Users\artyo\anaconda3\lib\site-packages\tensorflow_ranking\python\losses_impl.py", line 590, in _prepare_and_validate_params  *
        mask = utils.is_label_valid(labels)
    File "C:\Users\artyo\anaconda3\lib\site-packages\tensorflow_ranking\python\utils.py", line 75, in is_label_valid  *
        labels = tf.convert_to_tensor(value=labels)

    TypeError: Failed to convert elements of tf.RaggedTensor(values=Tensor("